# Workflow

In [21]:
# imports
import os

import numpy as np
import pandas as pd

In [22]:
# Paths to the files
filepath = os.getcwd()

price_app_filepath = filepath + "/datasets/announced-prices-apartments-luxembourg-city.xlsx"
price_house_filepath = filepath + "/datasets/announced-prices-houses-luxembourg-city.xlsx"
rent_ap_filepath = filepath + "/datasets/announced-rent-apartments-luxembourg-city.xlsx"
reg_price_filepath = filepath + "/datasets/registered-prices-apartements-by-commune.xlsx"

We open the dataset and start working on the indexing, organizing the data by Quarter and Year.
Since the first 3 datasets are similar we can create a function to avoid repeating code.

In [23]:
def clean_index(dataset):
    # ordering the data by quarter and year, creating a multi-index
    arrays = [[*dataset.Quarter], [*dataset.Year]]

    tuples = list(zip(*arrays))

    index =pd.MultiIndex.from_tuples(tuples, names=['Quarter', 'Year'])

    dataset.set_index(index, inplace=True)

    # we don't need the quarter and year since they are part  of the index
    dataset.drop(columns=['Quarter', 'Year'], inplace=True)

    dataset.sort_index(inplace=True)

    # we don't want to waste any data so we will return these info
    ret = [dataset.loc['Luxembourg City'], dataset.loc['National Average']]

    dataset.drop(index='Luxembourg City', inplace=True)
    dataset.drop(index='National Average', inplace=True)

    return ret

After we cleaned the data we check the datatype and we handle the missing data.
Different datasets may need different ways of handling, at first we will use `None` to replace `*`.

In [24]:
def check_type_missing(dataset, missing, rent=None):
    col1 = 'Number of offers'
    col2 = 'Average announced price in €'
    col3 = 'Average announced price per squared meter in €'

    if rent:
        col2 = 'Average announced rent in €'
        col3 = 'Average announced rent per squared meter in €'

    # TODO if missing replace with lux average

    # We handle the missing data replacing it with the average in the city of luxemburg
    # in the specific year
    for (commune, year) in dataset.index:
        if dataset.loc[(commune, year), col2] == '*':
            dataset.loc[(commune, year), col2] = missing[0].loc[year, col2]
        if dataset.loc[(commune, year), col3] == '*':
            dataset.loc[(commune, year), col3] = missing[0].loc[year, col3]

    # a safe check easy handling of missing data, may change for better modeling
    # val = 0
    # dataset.replace('*', val, inplace=True)


    # setting the type
    dataset.loc[:,col1] = dataset.loc[:,col1].astype('int64')
    dataset.loc[:,col2] = dataset.loc[:,col2].astype('float64').round(2)
    dataset.loc[:,col3] = dataset.loc[:,col3].astype('float64').round(2)

    # type check
    print(f"{col1 + ':':<50} \
        {str(dataset.loc[:,col1].dtype)}")
    print(f"{col2 + ':':<50} \
        {str(dataset.loc[:,col2].dtype)}")
    print(f"{col3 + ':':<50} \
        {str(dataset.loc[:,col3].dtype)}")

We aply the functions we've defined on the first 3 datasets.

In [25]:
# acquiring the data
price_ap_data = pd.read_excel(price_app_filepath)

price_ap_data

,Quarter,Number of offers,Average announced price in €,Average announced price per squared meter in €,Year
0,Beggen,495.0,364878,4222,2009
1,Belair,711.0,519909,5675,2009
2,Bonnevoie,804.0,323130,4124,2009
3,Cents,141.0,487993,5110,2009
4,Cessange,425.0,430093,4575,2009
...,...,...,...,...,...
320,Rollingergrund,152.0,960414.79,11234.22,2021
321,Ville-Haute,207.0,1000227.6,11740.79,2021
322,Weimerskirch,164.0,956075.03,16124.5,2021
323,Luxembourg City,NaN,1003203.5,12576.54,2021


In [26]:
# cleaning the indexing
temp =  clean_index(price_ap_data)
d1_lux_avg = temp[0]
d1_nat_avg = temp[1]

price_ap_data

Number of offers Average announced price in €  \
Quarter      Year                                                  
Beggen       2009             495.0                       364878   
             2010             508.0                       404496   
             2011             372.0                       422256   
             2012             160.0                       477997   
             2013             183.0                       500915   
...                             ...                          ...   
Weimerskirch 2017              29.0                            *   
             2018              33.0                    961181.82   
             2019               5.0                            *   
             2020              85.0                    1005641.4   
             2021             164.0                    956075.03   

                  Average announced price per squared meter in €  
Quarter      Year                                                 
Beggen       2009                                           4222  
             2010                                           4542  
             2011                                           5019  
             2012                                           5141  
             2013                                           5537  
...                                                          ...  
Weimerskirch 2017                                              *  
             2018                                       8335.125  
             2019                                              *  
             2020                                       15153.99  
             2021                                        16124.5  

[299 rows x 3 columns]

In [27]:
d1_nat_avg

,Number of offers,Average announced price in €,Average announced price per squared meter in €
Year,,,
2009,NaN,338330.06466307753,3876.9324903083739
2010,NaN,350562,3990
2011,NaN,364681.71109273488,4114.7719465322098
2012,NaN,372703.44050000003,4194.7179999999998
2013,NaN,392680.28579527879,4424.3909674434499
2014,NaN,401245.71436200634,4530.6284942842558
2015,NaN,459616.69660766964,5253.6683668754877
2016,NaN,485870.91974451084,5590.3031743067913
2017,NaN,499606.12560000003,5783.3565140000001


In [28]:
d1_lux_avg.loc[2017]

Number of offers                                                 NaN
Average announced price in €                      677440.51159999997
Average announced price per squared meter in €    8236.5313349999997
Name: 2017, dtype: object

In [29]:
# checking the types and handling missing values
check_type_missing(price_ap_data, temp)
price_ap_data

Number of offers:                                          int64
Average announced price in €:                              float64
Average announced price per squared meter in €:            float64


Number of offers  Average announced price in €  \
Quarter      Year                                                   
Beggen       2009               495                     364878.00   
             2010               508                     404496.00   
             2011               372                     422256.00   
             2012               160                     477997.00   
             2013               183                     500915.00   
...                             ...                           ...   
Weimerskirch 2017                29                     677440.51   
             2018                33                     961181.82   
             2019                 5                     839313.40   
             2020                85                    1005641.40   
             2021               164                     956075.03   

                   Average announced price per squared meter in €  
Quarter      Year                                                  
Beggen       2009                                         4222.00  
             2010                                         4542.00  
             2011                                         5019.00  
             2012                                         5141.00  
             2013                                         5537.00  
...                                                           ...  
Weimerskirch 2017                                         8236.53  
             2018                                         8335.12  
             2019                                        11257.35  
             2020                                        15153.99  
             2021                                        16124.50  

[299 rows x 3 columns]

In [30]:
# acquiring the data
price_hous_data = pd.read_excel(price_house_filepath)

# cleaning the indexing
temp = clean_index(price_hous_data)

d2_lux_avg = temp[0]
d2_nat_avg = temp[1]

# print(d2_lux_avg)
# print(d2_nat_avg)

price_hous_data.head()

Number of offers Average announced price in €  \
Quarter Year                                                  
Beggen  2009              74.0                       588378   
        2010              96.0                       618539   
        2011              76.0                       639310   
        2012              26.0           688769.23076923075   
        2013              28.0           763142.85714285716   

             Average announced price per squared meter in €  
Quarter Year                                                 
Beggen  2009                                           3515  
        2010                                           3761  
        2011                                           3873  
        2012                             3896.1231837816422  
        2013                             4170.1772762621586

In [31]:
# checking types and handling missing data
check_type_missing(price_hous_data, temp)
price_hous_data

Number of offers:                                          int64
Average announced price in €:                              float64
Average announced price per squared meter in €:            float64


Number of offers  Average announced price in €  \
Quarter      Year                                                   
Beggen       2009                74                     588378.00   
             2010                96                     618539.00   
             2011                76                     639310.00   
             2012                26                     688769.23   
             2013                28                     763142.86   
...                             ...                           ...   
Weimerskirch 2017                35                    1290428.57   
             2018                55                    1283943.30   
             2019                26                    1576213.60   
             2020                24                    1606490.20   
             2021                34                    1550529.40   

                   Average announced price per squared meter in €  
Quarter      Year                                                  
Beggen       2009                                         3515.00  
             2010                                         3761.00  
             2011                                         3873.00  
             2012                                         3896.12  
             2013                                         4170.18  
...                                                           ...  
Weimerskirch 2017                                         7023.61  
             2018                                         7432.01  
             2019                                         8078.17  
             2020                                         9527.81  
             2021                                         9612.59  

[299 rows x 3 columns]

In [32]:
# acquiring the data
rent_ap_data = pd.read_excel(rent_ap_filepath)

# cleaning the indexing
temp = clean_index(rent_ap_data)

d3_lux_avg = temp[0]
d3_nat_avg = temp[1]

print(d3_lux_avg)
print(d3_nat_avg)

rent_ap_data.head()

      Number of offers Average announced rent in €  \
Year                                                 
2009               NaN          1261.6096994337197   
2010               NaN                     1215.48   
2011               NaN          1304.3313996727622   
2012               NaN                     1331.89   
2013               NaN          1446.9852887090799   
2014               NaN          1433.0572939620972   
2015               NaN           1603.092977893368   
2016               NaN          1645.2385191846527   
2017               NaN          1705.5444440000001   
2018               NaN                   1857.5843   
2019               NaN          1771.6373000000001   
2020               NaN          1585.3983000000001   
2021               NaN                    1613.029   

     Average announced rent per squared meter in €  
Year                                                
2009                            17.420624143994186  
2010                          

Number of offers Average announced rent in €  \
Quarter Year                                                 
Beggen  2009             231.0                        1202   
        2010             324.0                        1093   
        2011             196.0                        1284   
        2012             107.0                        1261   
        2013              65.0                        1186   

             Average announced rent per squared meter in €  
Quarter Year                                                
Beggen  2009                                         15.19  
        2010                                         15.84  
        2011                            16.420000000000002  
        2012                                         17.02  
        2013                            19.239999999999998

In [33]:
check_type_missing(rent_ap_data, temp, rent=True)
rent_ap_data

Number of offers:                                          int64
Average announced rent in €:                               float64
Average announced rent per squared meter in €:             float64


Number of offers  Average announced rent in €  \
Quarter      Year                                                  
Beggen       2009               231                      1202.00   
             2010               324                      1093.00   
             2011               196                      1284.00   
             2012               107                      1261.00   
             2013                65                      1186.00   
...                             ...                          ...   
Weimerskirch 2017                26                      1705.54   
             2018                54                      1500.19   
             2019                87                      1415.40   
             2020                96                      1429.38   
             2021                75                      1435.20   

                   Average announced rent per squared meter in €  
Quarter      Year                                                 
Beggen       2009                                          15.19  
             2010                                          15.84  
             2011                                          16.42  
             2012                                          17.02  
             2013                                          19.24  
...                                                          ...  
Weimerskirch 2017                                          24.72  
             2018                                          26.76  
             2019                                          40.99  
             2020                                          39.01  
             2021                                          36.42  

[299 rows x 3 columns]

In [34]:
reg_price_data = pd.read_excel(reg_price_filepath)

In [35]:
tuples1 = []
# Sales of already constructed apartments
constructed = "Constructed"
# Sales of apartments still under construction (Ventes en Etat Futur D'Achevement [VEFA])
to_be_done = "VEFA"
# Price range for price per squared meter
r_min = "min range"
r_max = "max range"

for el in reg_price_data.iloc[0, 1:3]:
    tuples1.append((constructed, el))
tuples1.append((constructed, r_min))
#tuples1.append((constructed, r_max))

for el in reg_price_data.iloc[0, 1:3]:
    tuples1.append((to_be_done, el))
tuples1.append((to_be_done, r_min))
#tuples1.append((to_be_done, r_max))

print(tuples1)

new_header = reg_price_data.iloc[0]
reg_price_data.columns = new_header
reg_price_data = reg_price_data.iloc[1:]

reg_price_data

[('Constructed', 'Number of sales'), ('Constructed', 'Average registered price per squared meter in €'), ('Constructed', 'min range'), ('VEFA', 'Number of sales'), ('VEFA', 'Average registered price per squared meter in €'), ('VEFA', 'min range')]


,Commune,Number of sales,Average registered price per squared meter in €,Price range for price per squared meter,Number of sales,Average registered price per squared meter in €,Price range for price per squared meter,Year
1,Beaufort,3,*,*,0,*,*,2009
2,Bech,0,*,*,0,*,*,2009
3,Beckerich,3,*,*,0,*,*,2009
4,Berdorf,3,*,*,1,*,*,2009
5,Bertrange,27,4050.6390000000001,2478 € - 5653 €,23,5224.1819999999998,4027 € - 5745 €,2009
...,...,...,...,...,...,...,...,...
1368,Wiltz,26,4318.5469999999996,2904 € - 6132 €,39,4745.8869999999997,3753 € - 5622 €,2021
1369,Wincrange,5,*,*,4,*,*,2021
1370,Winseler,6,*,*,6,*,*,2021
1371,Wormeldange,18,5798.1880000000001,4003 € - 7510 €,8,*,*,2021


In [36]:
# ordering the data by Commune and year, creating a multi-index
arrays = [[*reg_price_data.loc[:,'Commune']], [*reg_price_data.loc[:,'Year']]]
tuples = list(zip(*arrays))

index =pd.MultiIndex.from_tuples(tuples, names=['Commune', 'Year'])
reg_price_data.set_index(index, inplace=True)

# we don't need the Commune and year since they are part  of the index
reg_price_data.drop(columns=['Commune', 'Year'], inplace=True)

reg_price_data.sort_index(inplace=True)

d4_nat_avg = reg_price_data.loc['National Average']
# print(d4_nat_avg)

reg_price_data.drop(index='National Average', inplace=True)

new_columns = pd.MultiIndex.from_tuples(tuples1, names=["Construction State", "Detail"])
reg_price_data.columns = new_columns

reg_price_data

Construction State     Constructed  \
Detail             Number of sales   
Commune     Year                     
Beaufort    2009                 3   
            2010                 2   
            2011                 4   
            2012                 8   
            2013                 1   
...                            ...   
Wormeldange 2017                 9   
            2018                11   
            2019                13   
            2020                14   
            2021                18   

Construction State                                                  \
Detail             Average registered price per squared meter in €   
Commune     Year                                                     
Beaufort    2009                                                 *   
            2010                                                 *   
            2011                                                 *   
            2012                                                 *   
            2013                                                 *   
...                                                            ...   
Wormeldange 2017                                                 *   
            2018                                4120.3959999999997   
            2019                                5201.6409999999996   
            2020                                          5848.607   
            2021                                5798.1880000000001   

Construction State                             VEFA  \
Detail                    min range Number of sales   
Commune     Year                                      
Beaufort    2009                  *               0   
            2010                  *               0   
            2011                  *               0   
            2012                  *               0   
            2013                  *               0   
...                             ...             ...   
Wormeldange 2017                  *               0   
            2018    3136 € - 4943 €               0   
            2019    3246 € - 8498 €              13   
            2020    4003 € - 7510 €              13   
            2021    4003 € - 7510 €               8   

Construction State                                                  \
Detail             Average registered price per squared meter in €   
Commune     Year                                                     
Beaufort    2009                                                 *   
            2010                                                 *   
            2011                                                 *   
            2012                                                 *   
            2013                                                 *   
...                                                            ...   
Wormeldange 2017                                                 *   
            2018                                                 *   
            2019                                5693.9480000000003   
            2020                                          7283.857   
            2021                                                 *   

Construction State                   
Detail                    min range  
Commune     Year                     
Beaufort    2009                  *  
            2010                  *  
            2011                  *  
            2012                  *  
            2013                  *  
...                             ...  
Wormeldange 2017                  *  
            2018                  *  
            2019    5014 € - 6660 €  
            2020    5934 € - 8671 €  
            2021                  *  

[1359 rows x 6 columns]

In [37]:
# function to clean separete correctly the min and max range in Price range for price per squared meter
def organize_range(ind1, ind2, to_insert, col_loc):
    aux = reg_price_data.loc[:,(ind1, ind2)].str.split(" - ", expand=True)
    col1_val = aux.iloc[:,0].map(lambda s: s.replace(" €", "") if s != None else "*")
    col2_val = aux.iloc[:,1].map(lambda s: s.replace(" €", "") if s != None else "*")
    # TODO #7 issue on split not good

    reg_price_data.loc[:,(ind1, ind2)] = col1_val
    reg_price_data.insert(col_loc, (ind1, to_insert), col2_val)

In [38]:
# splitting the constructed range
organize_range(constructed, r_min, r_max, 3)

# splitting the VEFA range
organize_range(to_be_done, r_min, r_max, 7)

reg_price_data


Construction State     Constructed  \
Detail             Number of sales   
Commune     Year                     
Beaufort    2009                 3   
            2010                 2   
            2011                 4   
            2012                 8   
            2013                 1   
...                            ...   
Wormeldange 2017                 9   
            2018                11   
            2019                13   
            2020                14   
            2021                18   

Construction State                                                            \
Detail             Average registered price per squared meter in € min range   
Commune     Year                                                               
Beaufort    2009                                                 *         *   
            2010                                                 *         *   
            2011                                                 *         *   
            2012                                                 *         *   
            2013                                                 *         *   
...                                                            ...       ...   
Wormeldange 2017                                                 *         *   
            2018                                4120.3959999999997      3136   
            2019                                5201.6409999999996      3246   
            2020                                          5848.607      4003   
            2021                                5798.1880000000001      4003   

Construction State                      VEFA  \
Detail             max range Number of sales   
Commune     Year                               
Beaufort    2009           *               0   
            2010           *               0   
            2011           *               0   
            2012           *               0   
            2013           *               0   
...                      ...             ...   
Wormeldange 2017           *               0   
            2018        4943               0   
            2019        8498              13   
            2020        7510              13   
            2021        7510               8   

Construction State                                                            \
Detail             Average registered price per squared meter in € min range   
Commune     Year                                                               
Beaufort    2009                                                 *         *   
            2010                                                 *         *   
            2011                                                 *         *   
            2012                                                 *         *   
            2013                                                 *         *   
...                                                            ...       ...   
Wormeldange 2017                                                 *         *   
            2018                                                 *         *   
            2019                                5693.9480000000003      5014   
            2020                                          7283.857      5934   
            2021                                                 *         *   

Construction State            
Detail             max range  
Commune     Year              
Beaufort    2009           *  
            2010           *  
            2011           *  
            2012           *  
            2013           *  
...                      ...  
Wormeldange 2017           *  
            2018           *  
            2019        6660  
            2020        8671  
            2021           *  

[1359 rows x 8 columns]

In [39]:
# todo #6 change type

reg_price_data.replace("*", "", inplace=True)

#reg_price_data.replace("*", None, inplace=True)
col1 = (constructed, "Number of sales")
col2 = (constructed, "Average registered price per squared meter in €")
col3 = (constructed, r_min)

print(f"{str(col1) + ':':<50} \
    {str(reg_price_data.loc[:,col1].dtype)}")
print(f"{str(col2) + ':':<50} \
    {str(reg_price_data.loc[:,col2].dtype)}")
print(f"{str(col3) + ':':<50} \
    {str(reg_price_data.loc[:,col3].dtype)}")

('Constructed', 'Number of sales'):                    int64
('Constructed', 'Average registered price per squared meter in €'):     object
('Constructed', 'min range'):                          object
